# Get predictions on test

In [7]:
import ray
from ray.train.sklearn import SklearnCheckpoint, SklearnPredictor

import pandas as pd
import numpy as np

## Configuration

In [9]:
INPUT_MODEL_PATH = '/Users/rgareev/projects/mlops-openfoodfacts/wrk/trainings/20220831-dev/model'
INPUT_DATA_PATH = '/Users/rgareev/data/openfoodfacts/wrk/20220831-dev/test.parquet'
# TODO this script should not deal with labels at all
LABEL_COLUMN = 'nova_group'

## Script

In [21]:
input_ds = ray.data.read_parquet(INPUT_DATA_PATH)

2022-09-11 17:06:47,030	WARNING read_api.py:291 -- ⚠️  The number of blocks in this dataset (1) limits its parallelism to 1 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.


In [22]:
input_ds = input_ds.repartition(10)

Repartition: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 14.32it/s]


In [23]:
from ray.data.context import DatasetContext

ctx = DatasetContext.get_current()
ctx.enable_tensor_extension_casting = False

In [24]:
input_ds = input_ds.drop_columns([LABEL_COLUMN])

Map_Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 13.72it/s]


In [3]:
model_checkpoint = SklearnCheckpoint.from_directory(INPUT_MODEL_PATH)

In [4]:
model = model_checkpoint.get_estimator()

In [6]:
model.steps

[('df_converter',
  FunctionTransformer(func=<function <lambda> at 0x133bdd790>)),
 ('encoder', DictVectorizer()),
 ('clf', BernoulliNB(binarize=None))]

In [31]:
model_out_list = [
    model.predict_proba(df)
    for df in input_ds.iter_batches(batch_size=None)
]

In [32]:
model_out = np.concatenate(model_out_list)

In [33]:
model_out.shape

(135135, 4)